In [1]:
from scipy.io import loadmat
import numpy as np

def load_mat_file(file_path):
    """
    Load a MATLAB .mat file and convert its contents to a Python dictionary
    with NumPy arrays.

    Parameters:
    file_path (str): The path to the .mat file.

    Returns:
    dict: A dictionary containing the data from the .mat file.
    """
    mat_data = loadmat(file_path)
    
    # Remove MATLAB-specific entries
    mat_data = {key: value for key, value in mat_data.items() if not key.startswith('__')}
    
    # Convert MATLAB arrays to NumPy arrays
    for key in mat_data:
        if isinstance(mat_data[key], np.ndarray):
            mat_data[key] = np.array(mat_data[key])
    
    return mat_data['val']

In [2]:
import wfdb

def load_hea_file(file_path):
    rd = wfdb.rdheader(file_path)
    
    return rd.fs, rd.sig_name, rd.n_sig, rd.units

In [3]:
import numpy as np
from scipy.signal import resample as sp_resample

def resample_eeg(eeg_data, fs_orig, fs_new):
    # eeg_data shape: (n_channels, n_samples)
    n_new = int(eeg_data.shape[1] * fs_new / fs_orig)
    return sp_resample(eeg_data, n_new, axis=1)


In [4]:
import mne
import numpy as np

def apply_eeg_filters(eeg, fs):
    # eeg shape: (n_channels, n_samples)
    # Scale normalized units to a range appropriate for MNE (convert to ~Volts range)
    eeg_scaled = eeg * 1e-6
    
    info = mne.create_info(
        ch_names=[f"ch{i}" for i in range(eeg_scaled.shape[0])],
        sfreq=fs,
        ch_types="eeg",
    )
    raw = mne.io.RawArray(eeg_scaled, info)  # no transpose; matches (channels, samples)
    raw.filter(l_freq=0.5, h_freq=50, fir_design="firwin")
    raw.notch_filter(freqs=60)
    
    # Scale back to original range
    return raw.get_data() * 1e6  # stays (n_channels, n_samples)

In [11]:
import numpy as np

def make_bipolar(eeg, ch_names):
    """
    eeg: (n_channels, n_samples) aligned to ch_names
    ch_names: list of labels in eeg row order
    returns: (bipolar_eeg, bipolar_names)
    """
    name_to_idx = {n: i for i, n in enumerate(ch_names)}

    pairs = [
        ("Fp1-F7", "Fp1", "F7"),
        ("F7-T3",  "F7",  "T3"),
        ("T3-T5",  "T3",  "T5"),
        ("T5-O1",  "T5",  "O1"),

        ("Fp2-F8", "Fp2", "F8"),
        ("F8-T4",  "F8",  "T4"),
        ("T4-T6",  "T4",  "T6"),
        ("T6-O2",  "T6",  "O2"),

        ("Fp1-F3", "Fp1", "F3"),
        ("F3-C3",  "F3",  "C3"),
        ("C3-P3",  "C3",  "P3"),
        ("P3-O1",  "P3",  "O1"),

        ("Fp2-F4", "Fp2", "F4"),
        ("F4-C4",  "F4",  "C4"),
        ("C4-P4",  "C4",  "P4"),
        ("P4-O2",  "P4",  "O2"),

        ("Fz-Cz",  "Fz",  "Cz"),
        ("Cz-Pz",  "Cz",  "Pz"),
        ("T3-C3",  "T3",  "C3"),  # optional alt: T3-C3 vs. T3-F7
    ]

    bipolar = []
    bipolar_names = []
    for new_name, a, b in pairs:
        ai, bi = name_to_idx[a], name_to_idx[b]
        bipolar.append(eeg[ai] - eeg[bi])
        bipolar_names.append(new_name)
    return np.vstack(bipolar), bipolar_names

# Usage:
# eeg_bipolar, bipolar_names = make_bipolar(eeg_resampled, ch_names)


In [5]:
eeg_unprocessed = load_mat_file("eeg_mat/0284_005_008_EEG.mat")
# print(eeg_unprocessed.shape)

In [10]:
fs_orig, ch_names, num_ch, units = load_hea_file("eeg_hea/0284_005_008_EEG")
print(ch_names)

['Fp1', 'Fp2', 'F3', 'F4', 'C3', 'C4', 'P3', 'P4', 'O1', 'O2', 'F7', 'F8', 'T3', 'T4', 'T5', 'T6', 'Fz', 'Cz', 'Pz']


In [7]:
new_fs = 200
eeg_resampled = resample_eeg(eeg_unprocessed, fs_orig, new_fs)
print(eeg_resampled.shape)

(19, 88600)


In [8]:
eeg_filtered = apply_eeg_filters(eeg_resampled, new_fs)

Creating RawArray with float64 data, n_channels=19, n_times=88600
    Range : 0 ... 88599 =      0.000 ...   442.995 secs
Ready.
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.5 - 50 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.50
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 0.25 Hz)
- Upper passband edge: 50.00 Hz
- Upper transition bandwidth: 12.50 Hz (-6 dB cutoff frequency: 56.25 Hz)
- Filter length: 1321 samples (6.605 s)

Filtering raw data in 1 contiguous segment
Setting up band-stop filter from 59 - 61 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband att

[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.0s


In [14]:
bp_eeg, bp_ch_names = make_bipolar(eeg_filtered, ch_names)

In [16]:
print(bp_eeg)
print(eeg_filtered)

[[-2.16840434e-12  7.97286003e+02  8.11324054e+02 ...  4.85420536e+02
  -3.18571505e+02 -8.03389244e-13]
 [ 3.90312782e-12 -1.61166988e+03 -1.37239827e+03 ...  1.98210175e+03
   1.54896548e+03  3.47147024e-12]
 [-5.20417043e-12  1.69690105e+03  1.40975814e+03 ... -2.03082323e+02
   7.43368846e-01 -1.76766512e-12]
 ...
 [-1.08420217e-12 -3.19558746e+03 -3.01424118e+03 ...  9.36828130e+02
   2.43253727e+02  1.61047433e-12]
 [ 0.00000000e+00  1.57960428e+03  1.50234323e+03 ... -3.10862146e+02
   2.89977988e+02 -3.33175116e-13]
 [-3.03576608e-12 -8.02826254e+02 -2.64332384e+03 ... -1.88608691e+03
  -1.65770062e+03 -3.59785847e-12]]
[[0.00000000e+00 2.27301993e+03 4.43114630e+03 ... 3.27595076e+03
  2.50805439e+03 2.41197926e-12]
 [5.42101086e-13 1.85465771e+03 3.77622510e+03 ... 3.49634208e+03
  2.74679874e+03 1.30413957e-12]
 [8.67361738e-13 2.57405384e+03 5.66727000e+03 ... 3.45341588e+03
  3.50541973e+03 1.48905745e-12]
 ...
 [0.00000000e+00 3.81340916e+03 6.92840905e+03 ... 3.83766269e